In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.applications.efficientnet as efn
import tensorflow as tf

In [2]:
train_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/TRAIN_GRAY_SCALE/'
valid_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/VALID_GRAY_SCALE/'
test_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/TEST_GRAY_SCALE/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

Found 55220 images belonging to 49 classes.


In [3]:
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=32)

Found 1728 images belonging to 49 classes.


In [4]:
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 1558 images belonging to 49 classes.


In [5]:
import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(49, activation='softmax')
])

# Build the model
model.build((None, 224, 224, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              4226432   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dense_1 (Dense)              (None, 49)                25137     
Total params: 4,907,441
Trainable params: 4,883,041
Non-trainable params: 24,400
_________________________________________________________________


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D



model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [7]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=5,
            verbose=1
)

Epoch 1/5
1726/1726 [==============================] - 12112s 7s/step - loss: 1.4350 - accuracy: 0.6653 - val_loss: 1.0139 - val_accuracy: 0.7772
Epoch 2/5
1726/1726 [==============================] - 6910s 4s/step - loss: 0.5980 - accuracy: 0.8934 - val_loss: 1.0892 - val_accuracy: 0.7749
Epoch 3/5
1726/1726 [==============================] - 6798s 4s/step - loss: 0.3822 - accuracy: 0.9580 - val_loss: 1.2325 - val_accuracy: 0.7836
Epoch 4/5
1726/1726 [==============================] - 6794s 4s/step - loss: 0.3266 - accuracy: 0.9760 - val_loss: 1.2738 - val_accuracy: 0.7865
Epoch 5/5
1123/1726 [==================>...........] - ETA: 39:25 - loss: 0.3047 - accuracy: 0.9823

KeyboardInterrupt: 

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [9]:
test_labels = test_batches.classes

In [10]:
predictions = model.predict(x=test_batches)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the testing accuracy
testing_accuracy = np.mean(predicted_labels == test_labels)

print("Testing Accuracy:", testing_accuracy)

Testing Accuracy: 0.7721437740693197


In [ ]:
def calculate_mean_average_precision(y_true, y_pred):
    num_classes = y_true.shape[1]
    average_precisions = []
    for class_idx in range(num_classes):
        class_true = y_true[:, class_idx]
        class_pred = y_pred[:, class_idx]
        sorted_indices = np.argsort(class_pred)[::-1]
        class_true_sorted = class_true[sorted_indices]
        true_positives = np.cumsum(class_true_sorted)
        num_positives = np.sum(class_true)
        recall = true_positives / num_positives
        precision = true_positives / np.arange(1, len(class_true) + 1)
        interpolated_precision = np.maximum.accumulate(precision)
        average_precision = np.sum((recall[1:] - recall[:-1]) * interpolated_precision[:-1])
        average_precisions.append(average_precision)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision


In [ ]:
test_labels = test_batches.classes

# Convert labels to one-hot encoding
num_classes = len(test_batches.class_indices)
one_hot_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

In [ ]:
# Calculate mAP
mAP = calculate_mean_average_precision(one_hot_labels, predictions)
print("Mean Average Precision (mAP):", mAP)

In [11]:
model.save_weights('MobileNetv3_Standford_GRAY.h5')

In [12]:
train_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/TRAIN/'
valid_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/VALID/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

Found 55220 images belonging to 49 classes.


In [13]:
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

Found 1728 images belonging to 49 classes.


In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [15]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=3,
            verbose=1
)

Epoch 1/3
1726/1726 [==============================] - 7446s 4s/step - loss: 0.3253 - accuracy: 0.9748 - val_loss: 1.4176 - val_accuracy: 0.7714
Epoch 2/3
 166/1726 [=>............................] - ETA: 1:51:36 - loss: 0.2801 - accuracy: 0.9910

KeyboardInterrupt: 

In [16]:
test_path = 'C:/Users/Prasanna P M/Human Project/STAND_THE_MAN/TEST/'

In [17]:
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 1558 images belonging to 49 classes.


In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [19]:
test_labels = test_batches.classes

In [20]:
predictions = model.predict(x=test_batches)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the testing accuracy
testing_accuracy = np.mean(predicted_labels == test_labels)

print("Testing Accuracy:", testing_accuracy)

Testing Accuracy: 0.8087291399229781


In [21]:
model.save_weights('MobileNetv3_Standford_RGB.h5')

In [22]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [23]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [24]:
TF_LITE_MODEL_FILE_NAME = "MobileNetv3_Standford_RGB.tflite"

In [25]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\PRASAN~1\AppData\Local\Temp\tmpsmly91bv\assets


INFO:tensorflow:Assets written to: C:\Users\PRASAN~1\AppData\Local\Temp\tmpsmly91bv\assets


In [26]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

5409104

In [27]:
import os
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

File size: 5.159 Megabytes
